In [ ]:
import copy
import math
import numpy as np

In [39]:
def f1(x):
    return x[0] - x[1] + 2*x[0]**2 + 2*x[0]*x[1]

def f2(x):
    return x[0]**2 + 2*x[1]**2 - 2*x[0]*x[1] - 2*x[1]

In [59]:
def nelder_mead(f, x_start,
                step=0.1, no_improve_thr=10e-4,
                no_improv_break=100, max_iter=0,
                alpha=1., gamma=2., rho=-0.5, sigma=0.5):
    """
        @param f (function): function to optimize, must return a scalar score
            and operate over a numpy array of the same dimensions as x_start
            
        @param x_start (numpy array): initial position
        
        @param step (float): look-around radius in initial step
        
        @no_improv_thr,  no_improv_break (float, int): break after no_improv_break iterations with
            an improvement lower than no_improv_thr
            
        @max_iter (int): always break after this number of iterations.
            Set it to 0 to loop indefinitely.
            
        @alpha, gamma, rho, sigma (floats): parameters of the algorithm
            (see Wikipedia page for reference)
            
        return: tuple (best parameter array, best score)
    """

    # INITIAL SIMPLEX & FUNCTION EVALUATION
    dim = len(x_start)
    prev_best = f(x_start)
    no_improv = 0
    res = [[x_start, prev_best]]

    for i in range(dim):
        x = copy.copy(x_start)
        x[i] = x[i] + step
        score = f(x)
        res.append([x, score])

    # SIMPLEX
    iters = 0
    while 1:
        # order
        res.sort(key=lambda x: x[1])
        best = res[0][1]
        
        
        # BREAK AFTER MAX_ITER
        if max_iter and iters >= max_iter:
            return res[0]
        iters += 1
        
        # BREAK WITH NO IMPROVEMENT
        print 'Iteration:',iters
        print  res[0][0],res[0][1]
        print  res[1][0],res[1][1]
        print  res[2][0],res[2][1]
        print 'Best Energy',best,'\n'
        
        if best < prev_best - no_improve_thr:
            no_improv = 0
            prev_best = best
        else:
            no_improv += 1

        if no_improv >= no_improv_break:
            return res[0]

        # CALCULATE CENTROID
        x0 = [0.] * dim
        for tup in res[:-1]:
            for i, c in enumerate(tup[0]):
                x0[i] += c / (len(res)-1)

        # REFLECTION
        xr = x0 + alpha*(x0 - res[-1][0])
        rscore = f(xr)
        if res[0][1] <= rscore < res[-2][1]:
            del res[-1]
            res.append([xr, rscore])
            continue

        # EXPANSION
        if rscore < res[0][1]:
            xe = x0 + gamma*(x0 - res[-1][0])
            escore = f(xe)
            if escore < rscore:
                del res[-1]
                res.append([xe, escore])
                continue
            else:
                del res[-1]
                res.append([xr, rscore])
                continue

        # CONTRACTION
        xc = x0 + rho*(x0 - res[-1][0])
        cscore = f(xc)
        if cscore < res[-1][1]:
            del res[-1]
            res.append([xc, cscore])
            continue

        # REDUCTION
        x1 = res[0][0]
        nres = []
        for tup in res:
            redx = x1 + sigma*(tup[0] - x1)
            score = f(redx)
            nres.append([redx, score])
        res = nres

In [62]:
if __name__ == "__main__":
    print nelder_mead(f2, np.array([-100.,-100.]))

Iteration: 1
[-100.   -99.9] 10179.82
[-100. -100.] 10200.0
[ -99.9 -100. ] 10200.01
Best Energy 10179.82 

Iteration: 2
[-100.   -99.9] 10179.82
[-100.1  -99.9] 10179.85
[-100. -100.] 10200.0
Best Energy 10179.82 

Iteration: 3
[-100.15  -99.7 ] 10139.6925
[-100.   -99.9] 10179.82
[-100.1  -99.9] 10179.85
Best Energy 10139.6925 

Iteration: 4
[-100.025  -99.6  ] 10119.540625
[-100.15  -99.7 ] 10139.6925
[-100.   -99.9] 10179.82
Best Energy 10119.540625 

Iteration: 5
[-100.2625  -99.15  ] 10030.2601563
[-100.025  -99.6  ] 10119.540625
[-100.15  -99.7 ] 10139.6925
Best Energy 10030.2601563 

Iteration: 6
[-100.13125  -98.725  ] 9946.05316406
[-100.2625  -99.15  ] 10030.2601563
[-100.025  -99.6  ] 10119.540625
Best Energy 9946.05316406 

Iteration: 7
[-100.540625  -97.6125  ] 9731.99907227
[-100.13125  -98.725  ] 9946.05316406
[-100.2625  -99.15  ] 10030.2601563
Best Energy 9731.99907227 

Iteration: 8
[-100.4828125  -96.20625  ] 9466.34402588
[-100.540625  -97.6125  ] 9731.99907227
[-1